In [1]:
#Importamos las librerías
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('once')


In [7]:
df=pd.read_csv('data/bikes_regis_estand_encod.csv',index_col=0)
df.head(3)

,registro,clima,temperatura,sens_termica,humedad,viento,registrados,fecha_nueva,festividad,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,año_map,estacion_map
0,1,2,-0.828948,-0.678635,1.295131,-0.376545,654,2018-01-01,New Year's Day,1,1,0,0,0,0
1,2,2,-0.722312,-0.740034,0.507407,0.882110,670,2018-01-02,NaN,0,1,1,1,0,0
2,3,1,-1.642047,-1.746977,-1.354487,0.861132,1229,2018-01-03,NaN,0,1,2,1,0,0


In [8]:
df.drop(['registro','fecha_nueva','festividad'], axis=1,inplace=True)
df.columns

Index(['clima', 'temperatura', 'sens_termica', 'humedad', 'viento',
       'registrados', 'festividad_1', 'mes_bueno', 'dia_semana_nuevo',
       'no_laboral_nuevo', 'año_map', 'estacion_map'],
      dtype='object')

In [9]:
X=df.drop(['registrados'],axis=1)
y=df['registrados']

In [11]:
# Separamos en train y test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Creamos objeto del modelo
arbol=DecisionTreeRegressor(random_state=0)
#Ajustamos el modelo
arbol.fit(x_train,y_train)

DecisionTreeRegressor(random_state=0)

In [12]:
# Obtenemos la raíz cuadrada del número de variables predictoras de nuestro árbol
max_features = np.sqrt(len(x_train.columns))
max_features

3.3166247903554

In [13]:
# Obtenemos la profundidad del árbol representado
arbol.tree_.max_depth

19

In [14]:
# Predicción para los datos de entrenamiento
y_pred_train1 = arbol.predict(x_train)
# Predicción para los datos de test
y_pred_test1 = arbol.predict(x_test)

In [15]:
def obtener_metricas(y_test_m, y_train_m, y_predict_test_m, y_predict_train_m, modelo):
    # En un diccionario, calcular las métricas de train y test
    resultados_metricas = {
        'MAE': [mean_absolute_error(y_test_m, y_predict_test_m), mean_absolute_error(y_train_m, y_predict_train_m)],
        'MSE': [mean_squared_error(y_test_m, y_predict_test_m), mean_squared_error(y_train_m, y_predict_train_m)],
        'RMSE': [np.sqrt(mean_squared_error(y_test_m, y_predict_test_m)), np.sqrt(mean_squared_error(y_train_m, y_predict_train_m))],
        'R2':  [r2_score(y_test_m, y_predict_test_m), r2_score(y_train_m, y_predict_train_m)],
        "set": ["test", "train"], 
        "modelo": [modelo, modelo]}

    # Pasamos a dataframe el diccionario para su mejor visualizacion
    df_resultados_metricas = pd.DataFrame(resultados_metricas)

    return df_resultados_metricas

In [16]:
df_resultados1= obtener_metricas(y_train,y_test,y_pred_train1,y_pred_test1,'Decision Tree Codif y Estand')
df_resultados1

,MAE,MSE,RMSE,R2,set,modelo
0,0.000000,0.000000,0.000000,1.000000,test,Decision Tree Codif y Estand
1,502.294521,648977.075342,805.591134,0.699349,train,Decision Tree Codif y Estand


In [18]:
# Definir diccionario con los hiperparámetros a modificar
# min_samples_split: número mínimo de observaciones que debe tener un nodo para dividirse, por defecto [10, 50, 100]
# min_samples_leaf: número mínimo de observaciones que debe tener cada uno de los nodos hijos para que se produzca la division, por defecto [10, 50, 100]
param = {"max_depth": [2,5,9], "max_features": [1,2,3], "min_samples_split": [10, 150, 300], "min_samples_leaf": [10,150,300]}

In [19]:
# Ajustar el modelo con GridSearchCV
# cv = 10 --> número de veces que hacemos la validación
# verbose = - 1 --> para no printear ningún mensaje en la pantalla
grid = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=param, cv=10, verbose=-1,
                        return_train_score=True, scoring="neg_mean_squared_error")

In [20]:
# Ajustar el modelo
grid.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [2, 5, 9], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 150, 300],
                         'min_samples_split': [10, 150, 300]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [21]:
# El método nos dice cuál es el mejor modelo,
mejor_modelo = grid.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=9, max_features=3, min_samples_leaf=10,
                      min_samples_split=10)

Siguientes pasos:
- obtener metricas
- modificar hiperparámetros
- desestandarizar para hacer decision tree solo con codificadas (o hacer en un momento dataframe con solo codificadas)
- realizar random forest
- comparar métricas